In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive
import dropbox
from amftrack.transfer.functions.transfer import upload
from subprocess import call

In [2]:
directory = directory_project
# directory = '/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/'

# update_plate_info(directory)

In [3]:
all_folders = get_current_folders(directory)
folders = all_folders.loc[all_folders['Plate']==792]

In [4]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp
5865,/projects/0/einf914/data/20210918_1729_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,True,False,True,True,True,True,"18.09.2021, 17:29:",20210918_1729_Plate21,NaN
5871,/projects/0/einf914/data/20210919_0129_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,True,False,True,True,True,True,"19.09.2021, 01:29:",20210919_0129_Plate21,NaN
5879,/projects/0/einf914/data/20211001_0323_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,True,False,True,True,True,False,"01.10.2021, 03:23:",20211001_0323_Plate21,NaN
5881,/projects/0/einf914/data/20210915_2318_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,True,False,True,True,True,True,"15.09.2021, 23:18:",20210915_2318_Plate21,NaN
5887,/projects/0/einf914/data/20210922_1320_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,True,False,True,True,True,True,"22.09.2021, 13:20:",20210922_1320_Plate21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,/projects/0/einf914/data/20210920_0329_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,True,False,True,True,True,True,"20.09.2021, 03:29:",20210920_0329_Plate21,NaN
7660,/projects/0/einf914/data/20211011_1353_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,True,False,True,True,True,False,"11.10.2021, 13:53:",20211011_1353_Plate21,NaN
7684,/projects/0/einf914/data/20210928_0308_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,True,False,True,True,True,False,"28.09.2021, 03:08:",20210928_0308_Plate21,NaN
7692,/projects/0/einf914/data/20210917_1318_Plate21,792,21,Carrot,A5,001P100N,Y,-,20210915,True,...,True,True,False,True,True,True,True,"17.09.2021, 13:18:",20210917_1318_Plate21,NaN


In [8]:
directory_targ = os.path.join(directory_scratch,'temp')+'/'
# directory_targ = os.path.join(directory_archive,'Prince_Data')+'/'


**To transfer data from surfsara to dropbox**
- use the 'toward_drop.py' function including the folders of interest and using dir_drop as a parameter indicating where in dropbox the data should go


In [7]:
dir_drop = 'prince_data'
run_parallel_transfer('toward_drop.py',[directory,dir_drop],folders,5,'20:00','staging',cpus = 1,node = 'staging')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248354
Submitted batch job 248355
Submitted batch job 248356
Submitted batch job 248357


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248358
Submitted batch job 248359
Submitted batch job 248360
Submitted batch job 248361
Submitted batch job 248362
Submitted batch job 248363
Submitted batch job 248364
Submitted batch job 248365
Submitted batch job 248366


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a n

Submitted batch job 248372
Submitted batch job 248373
Submitted batch job 248374
Submitted batch job 248375
Submitted batch job 248376


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248377
Submitted batch job 248378
Submitted batch job 248379
Submitted batch job 248380
Submitted batch job 248381
Submitted batch job 248382
Submitted batch job 248383


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


**To transfer data from dropbox to surfsara**
- use the 'from_drop.py' function including the folders of interest
- unzip argument decides if folders must also be unzipped
- flatten argument decides if folders must be flatten to respect snellius paralell computing logic or kept in a folder for transfer to archive for example

In [9]:
unzip = 'False'
flatten = False
run_parallel_transfer('from_drop.py',[directory_targ,dir_drop,unzip,flatten],folders,5,'10:00','staging',cpus = 1,node = 'staging')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248438
Submitted batch job 248439
Submitted batch job 248440
Submitted batch job 248441
Submitted batch job 248442
Submitted batch job 248443
Submitted batch job 248444
Submitted batch job 248445
Submitted batch job 248446
Submitted batch job 248447


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a n

Submitted batch job 248448
Submitted batch job 248449
Submitted batch job 248450
Submitted batch job 248451


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248460
Submitted batch job 248461
Submitted batch job 248462
Submitted batch job 248463


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248464
Submitted batch job 248465
Submitted batch job 248466
Submitted batch job 248467


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


**To transfer data from dropbox to archive**
- use the 'from_drop.py' function including the folders of interest


In [44]:
folders = all_folders.loc[all_folders['Plate'].isin([792,94])]

In [45]:
unzip = 'False'
flatten = False
directory_targ = os.path.join(directory_scratch,'temp')+'/'
directory = directory_targ
run_parallel_transfer('from_drop.py',[directory_targ,dir_drop,unzip,flatten],folders,5,'10:00','staging',cpus = 1,node = 'staging',name_job='archiving.sh')
run_parallel_transfer_to_archive(folders,directory,'10:00','staging',name_job='archiving.sh')

Submitted batch job 248604
Submitted batch job 248605
Submitted batch job 248606
Submitted batch job 248607
Submitted batch job 248608


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a n

Submitted batch job 248609
Submitted batch job 248610
Submitted batch job 248611
Submitted batch job 248612
Submitted batch job 248613


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248614
Submitted batch job 248615
Submitted batch job 248616
Submitted batch job 248617


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248664
Submitted batch job 248665
Submitted batch job 248666
Submitted batch job 248667


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 248668
Submitted batch job 248669


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [12]:
must_zip = True
must_unzip = False
run_parallel_transfer('within_surf.py',[directory,directory_targ,must_zip,must_unzip],folders,5,'10:00','staging',cpus = 1,node = 'staging')

Submitted batch job 242022


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [7]:
plates = set(folders['Plate'].values)

In [10]:
call(f"tar cvf /archive/cbisot/prince_data/94_20201123.tar {directory_targ}94_20201123*", shell=True)

tar: /archive/cbisot/prince_data/94_20201123.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


2

In [ ]:
def transfer_parallel_whole_plate(code, args, folders, num_parallel, time, name,cpus = 128,node = 'thin'):
    op_id = time_ns()
    folders.to_json(f'{directory_scratch}temp/{op_id}.json')# temporary file
    plates = set(folders['Plate'].values)
    length = len(plates)
    begin_skel = 0
    end_skel = length // num_parallel + 1
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    arg_str_out = "_".join([str(arg) for arg in args if type(arg)!=str])
    for j in range(begin_skel, end_skel):
        start = num_parallel * j
        stop = num_parallel * j + num_parallel - 1
        ide = time_ns()
        my_file = open(path_job, "w")
        my_file.write(
            f"#!/bin/bash \n#Set job requirements \n#SBATCH --nodes=1 \n#SBATCH -t {time}\n #SBATCH --ntask=1 \n#SBATCH --cpus-per-task={cpus}\n#SBATCH -p {node} \n"
        )
        my_file.write(
            f'#SBATCH -o "{path_code}slurm/{name}_{arg_str_out}_{start}_{stop}_{ide}.out" \n'
        )
        my_file.write(f"source /home/cbisot/miniconda3/etc/profile.d/conda.sh\n")
        my_file.write(f"conda activate amftrack\n")
        my_file.write(f"for i in `seq {start} {stop}`; do\n")
        my_file.write(f"\t python {path_code}amftrack/pipeline/scripts/image_processing/{code} {arg_str} {op_id} $i &\n")
        my_file.write("done\n")
        my_file.write("wait\n")
        my_file.close()
        call(f"sbatch {path_job}", shell=True)

In [8]:
def run_parallel_transfer(code, args, folders, num_parallel, time, name,cpus = 1,node = 'staging',name_job='transfer.sh'):
    path_job = f'{path_bash}{name_job}'
    op_id = time_ns()
    folders.to_json(f'{directory_scratch}temp/{op_id}.json')# temporary file
    length = len(folders)
    begin_skel = 0
    end_skel = length // num_parallel + 1
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    arg_str_out = "_".join([str(arg) for arg in args if type(arg)!=str])
    for j in range(begin_skel, end_skel):
        start = num_parallel * j
        stop = num_parallel * j + num_parallel - 1
        ide = time_ns()
        my_file = open(path_job, "w")
        my_file.write(
            f"#!/bin/bash \n#Set job requirements \n#SBATCH --nodes=1 \n#SBATCH -t {time}\n #SBATCH --ntask=1 \n#SBATCH --cpus-per-task={cpus}\n#SBATCH -p {node} \n"
        )
        my_file.write(
            f'#SBATCH -o "{path_code}slurm/{name}_{arg_str_out}_{start}_{stop}_{ide}.out" \n'
        )
        my_file.write(f"source /home/cbisot/miniconda3/etc/profile.d/conda.sh\n")
        my_file.write(f"conda activate amftrack\n")
        my_file.write(f"for i in `seq {start} {stop}`; do\n")
        my_file.write(f"\t python {path_code}amftrack/transfer/scripts/{code} {arg_str} {op_id} $i &\n")
        my_file.write("done\n")
        my_file.write("wait\n")
        my_file.close()
        call(f"sbatch {path_job}", shell=True)